In [1]:
# Importation de la librarie pymysql
import pymysql

In [2]:
# Connexion à la base de données
connection = pymysql.connect(
    host='localhost',
    user='root',
    database='ecole'
)

La connexion à la base de données se fait avec la fonction pymysql.connect(). Elle prend des paramètres comme l’hôte (host), l’utilisateur (user), et le nom de la base de données (database). Cette étape est essentielle car elle permet au programme de communiquer avec la base de données pour récupérer ou envoyer des informations. Une fois connecté, on peut exécuter des requêtes SQL pour obtenir les données des tables, comme les élèves et les enseignants dans cet exemple. La connexion est ensuite fermée à la fin pour libérer les ressources.

In [3]:
# Fonction pour récupérer les données des élèves et des enseignants
def fetch_data(cursor):
    # Données des élèves
    cursor.execute("SELECT prenom, nom, numero_classe FROM eleves")
    eleves = cursor.fetchall()

    # Données des enseignants
    cursor.execute("SELECT prenom, nom, numero_classe FROM enseignants")
    enseignants = cursor.fetchall()

    return eleves, enseignants


La fonction fetch_data sert à récupérer toutes les informations des élèves et des enseignants depuis la base de données. Elle envoie deux requêtes : une pour obtenir les élèves et l'autre pour obtenir les enseignants. On a besoin de ces données pour ensuite pouvoir les utiliser dans d'autres parties du programme.

In [4]:
# Fonction pour associer les élèves aux enseignants par numéro de classe
def associate_students_with_teachers(eleves, enseignants):
    associations = []
    for eleve in eleves:
        prenom_eleve, nom_eleve, classe_eleve = eleve
        enseignant_trouve = None

        for enseignant in enseignants:
            prenom_enseignant, nom_enseignant, classe_enseignant = enseignant
            if classe_eleve == classe_enseignant:
                enseignant_trouve = f"{prenom_enseignant} {nom_enseignant}"
                break

        associations.append((f"{prenom_eleve} {nom_eleve}", enseignant_trouve))
    return associations

La fonction associate_students_with_teachers permet d'associer chaque élève à son enseignant en fonction de leur numéro de classe. Elle parcourt la liste des élèves et compare leur numéro de classe avec celui des enseignants pour trouver le bon prof. Ça sert à créer un lien entre chaque élève et son prof responsable.

In [5]:
# Fonction pour compter le nombre d'élèves par enseignant
def count_students_per_teacher(eleves, enseignants):
    enseignant_eleves_count = {f"{prenom} {nom}": 0 for prenom, nom, _ in enseignants}

    for eleve in eleves:
        classe_eleve = eleve[2]
        for enseignant in enseignants:
            if classe_eleve == enseignant[2]:
                enseignant_nom_complet = f"{enseignant[0]} {enseignant[1]}"
                enseignant_eleves_count[enseignant_nom_complet] += 1
                break

    return enseignant_eleves_count

La fonction count_students_per_teacher compte combien d’élèves sont dans la classe de chaque enseignant. On commence par créer un dictionnaire où chaque enseignant a un compteur à 0, puis on vérifie à quelle classe appartient chaque élève et on augmente le nombre d'élèves du bon enseignant. Cela permet de savoir combien d'élèves sont sous la responsabilité de chaque enseignant.

In [6]:
# Fonction pour afficher les associations élèves-enseignants
def display_associations(associations):
    print("\nAssociations élèves-enseignants :")
    for eleve, enseignant in associations:
        if enseignant:
            print(f"{eleve} -> Enseignant responsable: {enseignant}")
        else:
            print(f"{eleve} -> Enseignant responsable: Non trouvé")

La fonction display_associations affiche la liste des élèves et de leurs enseignants. Elle parcourt la liste des associations créées et montre pour chaque élève le nom de son enseignant. Si un enseignant n'est pas trouvé, elle indique qu'aucun n'a été assigné. C'est utile pour visualiser qui est responsable de qui.

In [7]:
# Fonction pour afficher le nombre d'élèves par enseignant
def display_student_counts(enseignant_eleves_count):
    print("\nNombre d'élèves par enseignant :")
    for enseignant, count in enseignant_eleves_count.items():
        print(f"{enseignant} a {count} élève(s).")

La fonction display_student_counts sert à afficher le nombre d'élèves que chaque enseignant a dans sa classe. Elle parcourt le dictionnaire où on a compté les élèves et montre combien chaque enseignant en a. Ça permet de voir rapidement combien d'élèves chaque prof a sous sa responsabilité.

In [8]:
# Main function
def main():
    with connection:
        with connection.cursor() as cursor:
            # Extraction des données
            eleves, enseignants = fetch_data(cursor)

            # Association des élèves aux enseignants
            associations = associate_students_with_teachers(eleves, enseignants)

            # Comptage des élèves par enseignant
            enseignant_eleves_count = count_students_per_teacher(eleves, enseignants)

            # Affichage des résultats
            display_associations(associations)
            display_student_counts(enseignant_eleves_count)

La fonction principale gère tout le processus du programme. Elle commence par se connecter à la base de données, puis elle appelle les autres fonctions pour récupérer les données des élèves et des enseignants, associer les élèves à leurs enseignants et compter combien d’élèves chaque enseignant a. Ensuite, elle affiche les résultats. Enfin, elle ferme la connexion à la base de données. C'est comme un chef d'orchestre qui s'assure que chaque étape du programme est bien exécutée dans le bon ordre.

In [9]:
# Exécution du script
if __name__ == "__main__":
    main()


Associations élèves-enseignants :
Mark Watney -> Enseignant responsable: Salk John

Nombre d'élèves par enseignant :
Salk John a 1 élève(s).


La dernière clause "if" vérifie si le programme est exécuté directement (et pas importé dans un autre fichier). Si c'est le cas, elle lance la fonction main(), qui déclenche tout le reste du programme. Cela permet de s'assurer que le programme démarre correctement et exécute toutes les étapes, comme se connecter à la base de données, récupérer les infos, faire les associations et afficher les résultats.